In [8]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/Users/xingyuli/Desktop/Academics/chisel-bootcamp/source/load-ivy.sc"

In [9]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

In [10]:
// This is just a cell containing some constants
object TokenQueueConsts {
val ACETOKENS_PER_BIG_TOKEN = 3
val BDTOKENS_PER_BIG_TOKEN = 3
val BIG_TOKEN_WIDTH = 512
val TOKEN_QUEUE_DEPTH = 6144
}

defined object TokenQueueConsts

In [11]:
// Added for compiling unit tests
val NET_IF_WIDTH = 64
val ETH_MAC_BITS = 48

val RLIMIT_MAX_INC = 256
val RLIMIT_MAX_PERIOD = 256
val RLIMIT_MAX_SIZE = 256

class RateLimiterSettings extends Bundle {
  val incBits = log2Ceil(RLIMIT_MAX_INC)
  val periodBits = log2Ceil(RLIMIT_MAX_PERIOD)
  val sizeBits = log2Ceil(RLIMIT_MAX_SIZE)

  /*
   * Given a clock frequency of X, you can achieve an output bandwidth
   * of Y = X * (N / D), where N <= D, by setting inc to N and period to (D - 1).
   * The field size should be set to the number of consecutive beats that
   * can be sent before rate-limiting kicks in.
   */
  val inc = UInt(incBits.W)
  val period = UInt(periodBits.W)
  val size = UInt(sizeBits.W)
}

class PauserSettings extends Bundle {
  val threshold = UInt(16.W)
  val quanta    = UInt(16.W)
  val refresh   = UInt(16.W)
}


class StreamChannel(val w: Int) extends Bundle {
  val data = UInt(w.W)
  val keep = UInt((w/8).W)
  val last = Bool()
}

class NICIOvonly extends Bundle {
  val in = Flipped(Valid(new StreamChannel(NET_IF_WIDTH)))
  val out = Valid(new StreamChannel(NET_IF_WIDTH))
  val macAddr = Input(UInt(ETH_MAC_BITS.W))
  val rlimit = Input(new RateLimiterSettings)
  val pauser = Input(new PauserSettings)

}
// Added for compiling unit tests

NET_IF_WIDTH: Int = 64
ETH_MAC_BITS: Int = 48
RLIMIT_MAX_INC: Int = 256
RLIMIT_MAX_PERIOD: Int = 256
RLIMIT_MAX_SIZE: Int = 256
defined class RateLimiterSettings
defined class PauserSettings
defined class StreamChannel
defined class NICIOvonly

In [12]:
class NICTargetIO extends Bundle {
val clock = Input(Clock())
val nic = Flipped(new NICIOvonly)
}

defined class NICTargetIO

In [12]:
// // Un-comment when integration

// class NICBridge(implicit p: Parameters) extends BlackBox with Bridge[HostPortIO[NICTargetIO], SimpleNICBridgeModule] {
// val io = IO(new NICTargetIO)
// val bridgeIO = HostPort(io)
// val constructorArg = None
//   generateAnnotations()
// }
// object NICBridge {
// def apply(clock: Clock, nicIO: NICIOvonly)(implicit p: Parameters): NICBridge = {
// val ep = Module(new NICBridge)
//     ep.io.nic <> nicIO
//     ep.io.clock := clock
//     ep
//   }
// }

In [15]:
/* on a NIC token transaction:
 * 1) simulation driver feeds an empty token to start:
 *  data_in is garbage or real value (if exists)
 *  data_in_valid is 0 or 1 respectively
 *  data_out_ready is true (say host can always accept)
 *
 * 2) target responds:
 *  data_out garbage or real value (if exists)
 *  data_out_valid 0 or 1 respectively
 *  data_in_ready would be 1, so driver knows how to construct the next token if there was data to send
 *
 *  repeat
 */

// https://github.com/chipsalliance/rocket-chip/blob/master/src/main/scala/tilelink/Bundles.scala

In [15]:
// class ReadyValidLast extends Bundle {
//     val data_last = Bool()
//     val ready = Bool()
//     val valid = Bool()
// }
// class BIGToken extends Bundle {
//     val data = Vec(7, UInt(64.W))
//     val rvls = Vec(7, new ReadyValidLast())
//     val pad = UInt(43.W)
// }

defined class BIGToken

In [16]:
// class ACEToken extends Bundle{
//     val A = TLBundleA(params)
//     val C = TLBundleC(params)
//     val E = TLBundleE(params)
//     val Avalid = Bool()
//     val Cvalid = Bool()
//     val Evalid = Bool()
// }
// class BDToken extends Bundle{
//     val B = TLBundleB(params)
//     val D = TLBundleD(params)
//     val Bvalid = Bool()
//     val Dvalid = Bool()
// }

// class ACETokenGenerator extends Module {
//     val io = IO(new Bundle{
//         val in = Input(new ACEToken)
//         val out = Output(UInt(512.W))
//     })
//     io.out := io.in.asUInt
//     require(io.in.asUInt.getWidth <= 512)
// }

// class BDTokenGenerator extends Module {
//     val io = IO(new Bundle{
//         val in = Input(new BDToken)
//         val out = Output(UInt(512.W))
//     })
//     io.out := io.in.asUInt
//     require(io.in.asUInt.getWidth <= 512)
// }


In [83]:
class ACEToken extends Bundle{
    val A = UInt(8.W)
    val C = UInt(15.W)
    val E = UInt(9.W)
    val Avalid = Bool()
    val Cvalid = Bool()
    val Evalid = Bool()
}

defined class ACEToken

In [84]:
class ACETokenGenerator extends Module {
    val io = IO(new Bundle{
        val in = Input(new ACEToken)
        val out = Output(UInt(512.W))
    })
    io.out := io.in.asUInt
    require(io.in.asUInt.getWidth <= 512)
}
println(getVerilog(new ACETokenGenerator))

Elaborating design...
Done elaborating.
module ACETokenGenerator(
  input          clock,
  input          reset,
  input  [7:0]   io_in_A,
  input  [14:0]  io_in_C,
  input  [8:0]   io_in_E,
  input          io_in_Avalid,
  input          io_in_Cvalid,
  input          io_in_Evalid,
  output [511:0] io_out
);
  wire [34:0] _T = {io_in_A,io_in_C,io_in_E,io_in_Avalid,io_in_Cvalid,io_in_Evalid}; // @[cmd83.sc 6:21]
  assign io_out = {{477'd0}, _T}; // @[cmd83.sc 6:21]
endmodule



defined class ACETokenGenerator

In [87]:
test(new ACETokenGenerator) { c =>
    c.io.in.A.poke(1.U)
    c.io.in.C.poke(2.U)
    c.io.in.E.poke(3.U)
    c.io.in.Avalid.poke(true.B)
    c.io.in.Cvalid.poke(false.B)
    c.io.in.Evalid.poke(true.B)
    println(c.io.out.peek())
    c.clock.step(1)
    c.io.in.A.poke(0.U)
    c.io.in.C.poke(0.U)
    c.io.in.E.poke(0.U)
    c.io.in.Avalid.poke(true.B)
    c.io.in.Cvalid.poke(false.B)
    c.io.in.Evalid.poke(true.B)
    println(c.io.out.peek())
}

Elaborating design...
Done elaborating.
UInt<512>(134225949)
UInt<512>(5)
test ACETokenGenerator Success: 0 tests passed in 3 cycles in 0.005381 seconds 557.49 Hz


In [88]:
class ACETokenDecoder extends Module {
    val io = IO(new Bundle{
        val in = Input(UInt(512.W))
        val out = Output(new ACEToken)
    })
    io.out := io.in.asTypeOf(new ACEToken)
}
println(getVerilog(new ACETokenGenerator))

Elaborating design...
Done elaborating.
module ACETokenGenerator(
  input          clock,
  input          reset,
  input  [7:0]   io_in_A,
  input  [14:0]  io_in_C,
  input  [8:0]   io_in_E,
  input          io_in_Avalid,
  input          io_in_Cvalid,
  input          io_in_Evalid,
  output [511:0] io_out
);
  wire [34:0] _T = {io_in_A,io_in_C,io_in_E,io_in_Avalid,io_in_Cvalid,io_in_Evalid}; // @[cmd83.sc 6:21]
  assign io_out = {{477'd0}, _T}; // @[cmd83.sc 6:21]
endmodule



defined class ACETokenDecoder

In [94]:
test(new ACETokenDecoder) { c =>
    c.io.in.poke(134225949.U)
    println(c.io.out.A.peek())
    println(c.io.out.C.peek())
    println(c.io.out.E.peek())
    println(c.io.out.Avalid.peek())
    println(c.io.out.Cvalid.peek())
    println(c.io.out.Evalid.peek())
    
}

Elaborating design...
Done elaborating.
UInt<8>(1)
UInt<15>(2)
UInt<9>(3)
Bool(true)
Bool(false)
Bool(true)
test ACETokenDecoder Success: 0 tests passed in 2 cycles in 0.003595 seconds 556.27 Hz


In [94]:
// https://github.com/firesim/firesim/blob/366ceabe13ce27056ec1c53bf93d017ec182d85e/sim/midas/src/main/scala/midas/widgets/Widget.scala
// https://github.com/firesim/firesim/blob/366ceabe13ce27056ec1c53bf93d017ec182d85e/sim/midas/src/main/scala/junctions/nasti.scala


// class NastiIO(implicit p: Parameters) extends NastiBundle()(p) {
//   val aw = Decoupled(new NastiWriteAddressChannel)
//   val w  = Decoupled(new NastiWriteDataChannel)
//   val b  = Decoupled(new NastiWriteResponseChannel).flip
//   val ar = Decoupled(new NastiReadAddressChannel)
//   val r  = Decoupled(new NastiReadDataChannel).flip
// }


// class WidgetMMIO(implicit p: Parameters) extends NastiIO()(p)
//   with HasNastiParameters

// object WidgetMMIO {
//   def apply()(implicit p: Parameters): WidgetMMIO = {
//     new WidgetMMIO()(p alterPartial ({ case NastiKey => p(CtrlNastiKey) }))
//   }
// }

// // All widgets must implement this interface
// // NOTE: Changing ParameterizedBundle -> Bundle breaks PeekPokeWidgetIO when
// // outNum = 0
// class WidgetIO(implicit p: Parameters) extends ParameterizedBundle()(p){
//   val ctrl = Flipped(WidgetMMIO())
// }